In [1]:
import json
import requests
import subprocess
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio as rio
from rasterio.mask import mask
from dependencies import Box, GeoTIFF

def get_lambert(address:str) -> (int,int):
    req = requests.get(f"http://loc.geopunt.be/geolocation/location?q={address}&c=1")
    return (req.json()["LocationResult"][0]["Location"]["X_Lambert72"],
            req.json()["LocationResult"][0]["Location"]["Y_Lambert72"])

In [2]:
from geopandas import GeoDataFrame
from shapely.geometry import Polygon
from shapely.geometry import MultiPolygon
from BlenderObjectBuilder import Vertex
class Shape:
    @classmethod
    def poly_coord_to_vertex(cls, coord:(float,float)) -> Vertex:
        return Vertex(*coord)
    @classmethod
    def polygon_to_vertices(cls, poly:Polygon) -> [Vertex]:
        result = []
        for coord in poly.exterior.coords:
            result.append(cls.poly_coord_to_vertex(coord))
        return result[:-1]
    @classmethod
    def center(cls, poly:Polygon):
        return(int(poly.bounds[0]+poly.bounds[2]-poly.bounds[0]),
               int(poly.bounds[1]+poly.bounds[3]-poly.bounds[1]))
    @classmethod
    def mean_shape_height(cls, tif, shape):
        out_image, out_transform = mask(tif.load(), shapes=[shape], crop=True, invert=False)
        li = []
        for y in out_image[0]:
            for x in y:
                if x > 0: li.append(x)
        return np.array(li).mean()
    @classmethod
    def get_shape_entry_from_point(cls, x:float, y:float):
        return shape_lookup[shape_lookup.CaBl.apply(lambda s: Box.from_string(s).contains_point(x,y) == True)]
    @classmethod
    def get_shape_file_from_point(cls, x:float, y:float, ftype:str):
        entry = cls.get_shape_entry_from_point(x,y)
        return gpd.read_file(f"./Shapes/{entry.Province.values[0]}/{entry.Place.values[0]}/Bpn_{ftype}.shp")
    @classmethod
    def get_containing_shape_file(cls, tif_box:Box, ftype:str):
        entry = shape_lookup[shape_lookup[ftype].apply(
            lambda x:Box.from_string(x).contains_box(tif_box) if pd.notnull(x) else False)]
        if entry.empty: return False
        return gpd.read_file(f"./Shapes/{entry.Province.values[0]}/{entry.Place.values[0]}/Bpn_{ftype}.shp")
    @classmethod
    def get_shapes_in_box(cls, gdf: GeoDataFrame, box: Box) -> GeoDataFrame:
        return gdf[gdf.geometry.apply(
            lambda d: box.contains_point(
                x = cls.center(Polygon(d))[0],
                y = cls.center(Polygon(d))[1])
            if type(d) != MultiPolygon else False)]
shape_lookup = pd.read_csv("/media/becode/3D_House/Shapes/shape_lookup.csv", sep="|")

## Entire shapefile to blender (flat)

In [115]:
duffel = gpd.read_file("/media/becode/3D_House/Shapes/Antwerpen/Duffel/Bpn_CaBu.shp")

minx = duffel.bounds.minx.min()
miny = duffel.bounds.miny.min()
normalized_origin = Vertex(minx, miny)

data = {idx : Shape.polygon_to_vertices(entry)
        for idx, entry in duffel.geometry[
            duffel.geometry.apply(lambda x: type(x) != MultiPolygon)
        ].iteritems()}

for idx, shape in data.items():
    for i, vertex in enumerate(shape):
        data[idx][i] = (vertex -normalized_origin).scale(.01).xyz

with open("./object_data.json", "w") as out:
    json.dump(data, out)

## DSM, DTM & DCM to Blender

In [57]:
x, y = get_lambert(input("Address : "))

DSM = GeoTIFF.get_tif_from_point(x,y, "DSM").arr
DTM = GeoTIFF.get_tif_from_point(x,y, "DTM").arr
tifs = {"DSM": DSM, "DTM": DTM, "DCM": DSM - DTM}

data = {}
for ftype, tif in tifs.items():
    data[ftype] = {}
    for y in range(tif.shape[0]):
        data[ftype][y] = []
        for x in range(tif.shape[1]):
            data[ftype][y].append(np.float64(tif[y,x]))
        data[ftype][y] = data[ftype][y][::-1]
        
with open("./object_data.json", "w") as out:
    json.dump(data, out)

Address : Mechelsesteenweg 306 Sint-Kat


## Building heights

In [3]:
x, y = get_lambert(input("Address : "))
crop_DTM = GeoTIFF.get_containing_tif(x,y,100, "DTM")
crop_DSM = GeoTIFF.get_containing_tif(x,y,100, "DSM")
file = Shape.get_containing_shape_file(crop_DTM.box, "CaBu")
shapes = Shape.get_shapes_in_box(file, crop_DTM.box)
normalized_origin = Vertex(crop_DTM.box.left, crop_DSM.box.bottom)

terrain = {}
for idx, y in enumerate(range(crop_DTM.arr.shape[0])[::-1]):
    terrain[idx] = []
    for x in range(crop_DTM.arr.shape[1]):
        terrain[idx].append(np.float64(crop_DTM.arr[y,x]))      
blocks = {idx : {
    "vertices": list(map(lambda x: (x -normalized_origin).xyz, Shape.polygon_to_vertices(entry))),
    "height": np.float64(Shape.mean_shape_height(crop_DSM, entry) +1)}
        for idx, entry in shapes.geometry.iteritems()}
data = {"terrain": terrain, "blocks": blocks}

with open("./object_data.json", "w") as out:
    json.dump(data, out)

Address : Bosstraat 45 Duffel


In [9]:
shapes

,RecId,Type,FiscSitId,UpdDate,Shape_area,geometry
39,10021282,CL,2,2017-03-16,152.9999,"POLYGON ((157240.644 197556.888, 157251.642 19..."
117,2301550,CL,1,2016-12-01,108.7859,"POLYGON ((157400.871 197505.708, 157400.237 19..."
129,2301563,CL,1,2016-12-01,127.5275,"POLYGON ((157541.354 197643.111, 157530.358 19..."
130,2301564,CL,1,2016-12-01,20.4238,"POLYGON ((157521.786 197640.211, 157517.749 19..."
133,2301567,CL,1,2016-12-01,224.6420,"POLYGON ((157581.203 197664.594, 157585.134 19..."
...,...,...,...,...,...,...
9946,10220856,CL,3,2018-01-25,47.9999,"POLYGON ((157362.967 197476.776, 157368.961 19..."
9947,10750989,CL,5,2019-10-01,130.9005,"POLYGON ((157553.182 197495.264, 157560.874 19..."
9948,10750856,CL,5,2019-10-01,148.4598,"POLYGON ((157599.987 197492.043, 157608.278 19..."
9961,10798986,UN,5,2020-01-08,257.5274,"POLYGON ((157066.232 197806.036, 157073.495 19..."


In [68]:
x, y = get_lambert(input("Address : "))
crop_DTM = GeoTIFF.get_containing_tif(x,y,100, "DTM")
crop_DSM = GeoTIFF.get_containing_tif(x,y,100, "DSM")
file = Shape.get_containing_shape_file(crop_DTM.box, "CaBu")
shapes = Shape.get_shapes_in_box(file, crop_DTM.box)
normalized_origin = Vertex(crop_DTM.box.left, crop_DSM.box.bottom)

out_image, out_transform = mask(
    crop_DSM.load(), shapes=shapes.geometry,pad=True,pad_width=2, crop=False, invert=True, nodata=crop_DTM.arr.mean())

data = {}
data["test"] = {}
for idx, y in enumerate(range(out_image[0].shape[0])[::-1]):
    data["test"][idx] = []
    for x in range(out_image[0].shape[1]):
        data["test"][idx].append(np.float64(out_image[0][y,x]))
    #data["test"][idx] = data["test"][idx][::-1]
with open("./object_data.json", "w") as out:
    json.dump(data, out)

Address : Bosstraat 45 Duffel


## Inject Blender

In [4]:
subprocess.run("blender -P blender_inject_template.py", shell=True)

CompletedProcess(args='blender -P blender_inject_template.py', returncode=0)